In [1]:
#!/usr/bin/env python

from scipy import *
from scipy.optimize import *
import numpy as np

#Define all the functions here-

#ALL FUNCTIONS FOR DIRECT TITRATION
#Fit function for Direct Titration
def fit_direct(Ht, k, dF, IT_D):
    #Fbound = min(dF)
    #Ffree = max(dF)
    b=(k+IT_D+Ht)
    return (dF)*(b-sqrt((b**2.0)-(4.0*Ht*IT_D)))/(2.0*IT_D)

#The two err functions
def err_direct(param, X, Y, const):
    yi=fit_direct(X, param[0], param[1], const[0])
    return sum(abs(yi-Y)**2)

def err_direct_lsq(param, X, Y, const):
    yi=fit_direct(X, param[0], param[1], const[0])
    return abs(Y-yi)

def fit_dir_wrap(arg, ki, df):
    #this is for curve_fit so that other arguments can be passed to fit_comp
    x, const = arg
    yi= fit_direct(x, ki, df, const[0])
    return yi

#ALL FUNCTIONS FOR COMPTETIVE TITRATIONS
#Fit function for comptetive titration
def fit_comp(GT, KG, EI, KI, ITc, HTc):
    #Calculate a,b,c - coefficients of the cubic equation
    a=KG+KI+ITc-HTc+GT
    b=(KG*KI)+(KI*(GT-HTc))+(KG*(ITc-HTc))
    c=-(KI*KG*HTc)
    #Calculate q,r - coefficients of reduced cubic equation 
    r=(-c)+(a*b/3.0)+(-2.0*(a**3)/27.0)
    q=((a**2)/3.0)-b
    #Calculate n, theta - for trignometric solution of reduced cubic equation (By comparing with equation for cos3A)
    n=2*sqrt(q/3.0)
    theta=arccos((4.0*r)/(n**3))/3.0
    #Solution of reduced cubic equation
    A = n*cos(theta)
    #Solution of initial cubic equation ([H] free)
    host = A-(a/3.0)
    #Fit function
    ans = (EI*(1.0-(host/(KI+host))))
    return ans-min(ans)
    #return ans

#The two error functions
def err_comp(params, X, Y, const):
    yi= fit_comp(X, params[0], params[1], const[0], const[1], const[2])
    return sum(abs(Y-yi)**2.0)

def err_comp_lsq(X, Y, params, const):
    yi= fit_comp(X, params[0], params[1], const[0], const[1], const[2])
    return abs(Y-yi)

def fit_comp_wrap(arg, kg, ei):
    #this is for curve_fit so that other arguments can be passed to fit_comp
    x,const = arg 
    yi= fit_comp(x, kg, ei, const[0], const[1], const[2])
    return yi

#Calculation of R2
def Rsq(X, Y, sol, const, fun):
    ssres = abs(fun(sol, X, Y, const))
    ybar=sum(Y)/(len(Y))
    sstot = sum((Y-ybar)**2)
    r=1.0-(ssres/sstot)
    return r
